In [2]:
import numpy as np
import math

In [3]:
# random_sequence = np.random.randint(1000,2000, size = 10)
random_sequence = np.array([1896,   1026,   1367,   1389,  1365,  1752,   1651,   1380,   1132,   1437])
# append_random_sequence = np.append(random_sequence,np.random.randint(1000,2000, size = 200))
# append_random_sequence = np.append(random_sequence,[1000,900,200,1500,159])
# append_random_sequence = np.append(random_sequence,[10,90,20,150,159])


In [4]:
append_random_sequence = np.append(random_sequence,np.random.randint(1000,2000, size = 200))
# append_random_sequence = np.append(random_sequence,[1000,900,200,1500,159])
# append_random_sequence = np.append(random_sequence,[10,90,20,150,159])


In [5]:
def update(p, x_n, current_quantile, seq_size, max_gap_record, min_gap_record, min_gap, c_minus, c_plus, x_max, x_min):
    max_value = max_gap_record[0]
    max_prev_value = max_gap_record[1]
    min_value = min_gap_record[0]
    min_prev_value = min_gap_record[1]
    
    lambda_x = min_gap
    if x_n < current_quantile:
        floor_value = math.floor(seq_size*p)
        round_value = round(seq_size*p)
        if floor_value % 2 == 0 and seq_size*p - floor_value == 0.5:
            round_value = round(seq_size*p) + 1
        print("c_minux+1: {}, seq_size*p: {}, round value: {}".format(c_minus + 1, seq_size * p, round_value))
        if c_minus + 1 >= seq_size * p:
            if current_quantile == min_prev_value:
                current_quantile = min_value
                min_gap_record = [x_min, min_value]
            elif current_quantile == x_max:
                current_quantile = max_value
                x_max = max_value
                print("*****", max_gap_record, min_gap_record)
                # if x_n < max_prev_value:
                #     if x_n > min_prev_value:
                #         max_gap_record = [max_prev_value, x_n]
                #     else:
                #         max_gap_record = [max_prev_value,max_prev_value - lambda_x]
                # else:
                #     max_gap_record = [x_n,max_prev_value]
            else:
                print("============= minus", current_quantile, lambda_x)
                current_quantile = current_quantile - lambda_x
            c_plus = c_plus + 1
        else:
            c_minus = c_minus + 1
    elif x_n >= current_quantile:
        print("c_plus+1: {}, seq_size*(1-p): {}".format(c_plus + 1, seq_size * (1-p)))
        if c_plus + 1 > (seq_size * (1 - p)):
            if current_quantile == max_prev_value:
                current_quantile = max_value
                max_gap_record = [x_max, max_value]
                print("!!!!!!!max_gap_record", max_gap_record)
            else:
                print("~~~~~~~~~~~ pluss", current_quantile, lambda_x)

                current_quantile = current_quantile + lambda_x
            print("current_quantile", current_quantile)
            c_minus = c_minus + 1
        else:
            c_plus = c_plus + 1
    return current_quantile, c_minus, c_plus, max_gap_record, min_gap_record,x_max, x_min


In [6]:
p = 0.9
for  i in range(len(append_random_sequence)):
    x_n = append_random_sequence[i]
    seq_size = i + 1
    if i == 0:
        c_minus = 0
        c_plus = 0
        current_quantile = x_n
        x_prev_max = x_n
        x_prev_min = x_n
        x_max = x_n
        x_min = x_n
        continue
    elif i == 1 or i == 2:
        # min_gap = [-1,-1]
        min_gap = abs(x_n - x_prev_max)
        if x_n < x_min:
            x_prev_min = x_min
            x_min = x_n
            record_min_gap = [x_min,x_prev_min]
        else:
            x_prev_min = x_n
            record_min_gap = [x_min, x_prev_min]
        if x_n > x_max:
            x_prev_max = x_max
            x_max = x_n
            record_max_gap = [x_max,x_prev_max]
        else:
            x_prev_max = x_n
            record_max_gap = [x_max, x_prev_max]
    # elif i == 2:
    #     print("i==2", x_n, x_prev_max, x_max, x_prev_min, x_min)
    else:
        x_prev_max = record_max_gap[1]
        x_prev_min = record_min_gap[1]
        print("@@@@@@@",i, record_max_gap, x_prev_max)
        
        gap = abs(x_n - x_prev_max)
        if min_gap> gap and gap > 0:
            min_gap = gap
        if x_n > x_max:
            print("--- x_n:{}, x_prev_max:{}, x_max:{}".format(x_n, x_prev_max, x_max))
            gap_before = x_max - x_prev_max
            gap_after = x_n - x_max
            if gap_after > gap_before:
                x_prev_max = x_max
                x_max = x_n
                record_max_gap = [x_max, x_prev_max]
            x_max = x_n
        elif x_n < x_min:
            print("++= x_n:{}, x_prev_min:{}, x_min: {}".format(x_n, x_prev_min, x_min))
            gap_before = x_prev_min - x_min
            gap_after = x_min - x_n
            if gap_after > gap_before:
                x_prev_min = x_min
                x_min = x_n
                record_min_gap = [x_min, x_prev_min]
            x_min = x_n
        elif x_n > x_prev_max and x_n < x_max:
            gap_before = x_n - x_prev_max
            gap_after = record_max_gap[0] - x_n
            if gap_after > gap_before:
                print("----------------k gap_before: [{},{}] ; gap_after: [{},{}]".format(x_prev_max, x_n, x_n, record_max_gap[0]), x_n,x_prev_max)
                x_prev_max = x_n
                record_max_gap = [record_max_gap[0], x_n]
            else:
                print("j",x_n)
                if x_max == current_quantile:
                    if record_max_gap[0] != x_max:
                       record_max_gap = [record_max_gap[0], x_n]
                    else:
                        record_max_gap = [x_n, x_prev_max]
        elif x_n < x_prev_min and x_n > x_min:
            gap_before = x_n - record_min_gap[0]
            gap_after = x_prev_min - x_n
            if gap_after > gap_before:
                print("jjjj", x_n, x_prev_min)
                record_min_gap = [x_n, x_prev_min]
            else:
                print("tk gap_before: [{},{}]; gap_after: [{},{}]".format(x_n, record_min_gap[0], x_n, x_prev_min), x_n, x_prev_max)
                x_prev_min = x_n
                record_min_gap = [record_min_gap[0], x_n]
        print("record_max_gap:{}, x_prev_max: {}, x_max: {}, x_n:{}".format( record_max_gap, x_prev_max, x_max, x_n))
        print("record_min_gap:{}, x_prev_min: {}, x_min: {}, x_n:{}".format( record_min_gap, x_prev_min, x_min, x_n))

    print("Before Round% {},current_quantile:{},  x_n:{}, record_min_gap: {}, record_max_gap:{}, min_gap:{}, c_minus:{}, c_plus:{} , x_max:{},x_min:{}".format(i,current_quantile, x_n,record_min_gap, record_max_gap,  min_gap, c_minus, c_plus,x_max,x_min), seq_size )
    current_quantile, c_minus, c_plus, record_max_gap, record_min_gap,x_max, x_min = update(p, x_n, current_quantile,seq_size, record_max_gap, record_min_gap, min_gap, c_minus, c_plus, x_max, x_min)
    # x_prev_max = x_n
    print("After Round% {}, current_quantile:{}, record_min_gap: {}, record_max_gap:{}, min_gap:{}, c_minus:{}, c_plus:{}, x_max:{},x_min:{}".format(i, current_quantile, record_min_gap, record_max_gap,  min_gap, c_minus, c_plus,x_max,x_min)  )

Before Round% 1,current_quantile:1896,  x_n:1026, record_min_gap: [1026, 1896], record_max_gap:[1896, 1026], min_gap:870, c_minus:0, c_plus:0 , x_max:1896,x_min:1026 2
c_minux+1: 1, seq_size*p: 1.8, round value: 2
After Round% 1, current_quantile:1896, record_min_gap: [1026, 1896], record_max_gap:[1896, 1026], min_gap:870, c_minus:1, c_plus:0, x_max:1896,x_min:1026
Before Round% 2,current_quantile:1896,  x_n:1367, record_min_gap: [1026, 1367], record_max_gap:[1896, 1367], min_gap:341, c_minus:1, c_plus:0 , x_max:1896,x_min:1026 3
c_minux+1: 2, seq_size*p: 2.7, round value: 3
After Round% 2, current_quantile:1896, record_min_gap: [1026, 1367], record_max_gap:[1896, 1367], min_gap:341, c_minus:2, c_plus:0, x_max:1896,x_min:1026
@@@@@@@ 3 [1896, 1367] 1367
----------------k gap_before: [1367,1389] ; gap_after: [1389,1896] 1389 1367
record_max_gap:[1896, 1389], x_prev_max: 1389, x_max: 1896, x_n:1389
record_min_gap:[1026, 1367], x_prev_min: 1367, x_min: 1026, x_n:1389
Before Round% 3,curre

In [7]:
print(append_random_sequence)
sorted_random_sequence = sorted(append_random_sequence)
print(sorted_random_sequence)
print("totel len: {}, p90 len: {}".format(len(sorted_random_sequence),int(0.9 * len(sorted_random_sequence))))
p90_value = sorted_random_sequence[int(0.9 * len(sorted_random_sequence))]
print(p90_value)

[1896 1026 1367 1389 1365 1752 1651 1380 1132 1437 1120 1645 1559 1530
 1081 1821 1392 1807 1632 1928 1144 1473 1898 1837 1131 1730 1783 1971
 1566 1262 1671 1442 1873 1259 1272 1227 1251 1671 1317 1689 1054 1931
 1381 1166 1108 1675 1006 1450 1941 1399 1070 1758 1775 1162 1596 1788
 1090 1506 1273 1721 1805 1666 1150 1245 1525 1716 1205 1355 1923 1367
 1119 1093 1709 1366 1268 1100 1743 1498 1291 1658 1973 1084 1855 1514
 1584 1406 1862 1499 1438 1359 1137 1784 1746 1641 1914 1078 1732 1117
 1140 1690 1493 1798 1140 1396 1711 1068 1738 1213 1609 1303 1171 1030
 1993 1388 1526 1908 1664 1720 1115 1295 1718 1757 1530 1254 1770 1440
 1006 1268 1996 1203 1778 1714 1390 1408 1826 1986 1134 1122 1423 1760
 1357 1676 1180 1297 1254 1666 1580 1477 1155 1954 1973 1929 1080 1825
 1835 1334 1821 1355 1492 1284 1862 1821 1370 1466 1391 1524 1698 1459
 1593 1428 1489 1512 1077 1184 1357 1813 1696 1840 1074 1079 1010 1695
 1789 1656 1803 1369 1101 1990 1105 1464 1943 1115 1408 1353 1037 1242
 1565 